# Installing required packages

In [1]:
%conda install --file requirements.txt

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - libwebp==1.2.4=h11a3e52_1
  - protobuf==3.19.6=pypi_0
  - libzopfli==1.0.3=he6710b0_0
  - tensorflow-io-gcs-filesystem==0.31.0=pypi_0
  - libcurl==7.88.1=h91b91d3_0
  - pyasn1-modules==0.2.8=pypi_0
  - pyyaml==6.0=py39h5eee18b_1
  - gdk-pixbuf==2.42.10=h5eee18b_0
  - numpy-base==1.23.5=py39h31eccc5_0
  - pytz==2022.7=py39h06a4308_0
  - tensorflow-estimator==2.10.0=pypi_0
  - certifi==2022.12.7=py39h06a4308_0
  - libgcc-ng==11.2.0=h1234567_1
  - charls==2.2.0=h2531618_0
  - tensorboard-plugin-wit==1.8.1=pypi_0
  - jpeg==9e=h166bdaf_1
  - zstd==1.5.2=ha4553b6_0
  - debugpy==1.5.1=py39h295c915_0
  - markupsafe==2.1.2=pypi_0
  - krb5==1.19.4=h568e23c_0
  - pyzmq==19.0.2=py39hb69f2a1_2
  - cfitsio==3.470=h5893167_7
  - joblib==1.1.1=py39h06a4308_0
  - python==3.9.16=h7a1cb2a_2
  - cairo==1.16.0=hb05425b_4
  - astunparse==1.6.3=pypi_0
  - numpy==1.24.2=pypi_0
  - tk==8.6.12=h1ccaba5_0
  - cffi==1.15.

# Importing tensorflow and creating datasets from directory

In [4]:
%conda install -c conda-forge tensorflow

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf

train_data_dir = 'food20dataset/train_set'
test_data_dir = 'food20dataset/test_set'
img_width, img_height = 200, 150
batch_size = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

Found 1400 images belonging to 20 classes.
Found 600 images belonging to 20 classes.


In [6]:
import os
from PIL import Image

train_data_dir = 'food20dataset/train_set'

min_width = float('inf')
min_height = float('inf')
for dirpath, dirnames, filenames in os.walk(train_data_dir):
    for filename in filenames:
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            img = Image.open(os.path.join(dirpath, filename))
            width, height = img.size
            if width < min_width:
                min_width = width
            if height < min_height:
                min_height = height

print("Minimum width:", min_width)
print("Minimum height:", min_height)
num_classes = train_generator.num_classes

Minimum width: 100
Minimum height: 100


# Model and training

In [7]:
# 7 epochs is only required for 100% accuracy
num_epochs = 7

import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


C:\Users\somanathan\AppData\Local\Temp\ipykernel_5472\391245141.py:16: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/7
43/43 [==============================] - 91s 2s/step - loss: 7.2634 - accuracy: 0.0556 - val_loss: 2.9745 - val_accuracy: 0.0816
Epoch 2/7
43/43 [==============================] - 85s 2s/step - loss: 2.8187 - accuracy: 0.1367 - val_loss: 2.6410 - val_accuracy: 0.2448
Epoch 3/7
43/43 [==============================] - 69s 2s/step - loss: 2.0679 - accuracy: 0.4174 - val_loss: 2.1526 - val_accuracy: 0.3611
Epoch 4/7
43/43 [==============================] - 96s 2s/step - loss: 1.0191 - accuracy: 0.7500 - val_loss: 1.9084 - val_accuracy: 0.4861
Epoch 5/7
43/43 [==============================] - 82s 2s/step - loss: 0.3975 - accuracy: 0.9313 - val_loss: 1.8603 - val_accuracy: 0.5191
Epoch 6/7
43/43 [==============================] - 90s 2s/step - loss: 0.1429 - accuracy: 0.9861 - val_loss: 1.9767 - val_accuracy: 0.5035
Epoch 7/7
43/43 [==============================] - 87s 2s/step - loss: 0.0578 - accuracy: 0.9985 - val_loss: 1.8299 - val_accuracy: 0.5260


# saving model for later use

In [8]:
model.save(filepath='food')

INFO:tensorflow:Assets written to: food\assets


INFO:tensorflow:Assets written to: food\assets


# loading the same model

In [9]:
m = tf.keras.models.load_model('food')

# Food index and prediction

In [10]:
food_index = {
    0:'biriyani',
    1:'bisibelebath',
    2:'butternaan',
    3:'chaat',
    4:'chappati',
    5:'dhokla',
    6:'dosa',
    7:'gulab jamun',
    8:'halwa',
    9:'idly',
    10:'kathi roll',
    11:'meduvadai',
    12:'noodles',
    13:'paniyaram',
    14:'poori',
    15:'samosa',
    16:'tandoori chicken',
    17:'upma',
    18:'vada pav',
    19:'ven pongal'
                }

In [13]:
# test image in same folder as notebook with name 'testSample.jpg'

i = Image.open('testSample.jpg').resize((150,200))
i = tf.keras.preprocessing.image.img_to_array(i)
prediction = m.predict(tf.expand_dims(i, axis=0))[0]
predicted_class_index = tf.argmax(prediction).numpy()
print(food_index[predicted_class_index])


1/1 [==============================] - 0s 37ms/step
gulab jamun
